<a href="https://colab.research.google.com/github/hivani/-Question-Answering-system-using-an-LLM-/blob/main/steps_AI_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Advanced Chunking of Data***

In [1]:
pip install sentence-transformers

In [2]:
import json
from sentence_transformers import SentenceTransformer, util

# Load JSON Data
with open('/content/scraped_data.json') as f:
    data = json.load(f)

# Initialize the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to chunk data based on semantic similarity
def chunk_data(doc):
    sentences = doc.split('. ')
    embeddings = model.encode(sentences)
    clusters = util.community_detection(embeddings, min_community_size=2)
    chunks = [' '.join([sentences[idx] for idx in cluster]) for cluster in clusters]
    return chunks

# Chunk all documents
all_chunks = []
for entry in data:
    chunks = chunk_data(entry['content'])
    for chunk in chunks:
        all_chunks.append({
            'text': chunk,
            'meta': {'web_link': entry.get('web_link', 'N/A')}
        })

print(f"Total chunks created: {len(all_chunks)}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

Total chunks created: 5


In [3]:
from bs4 import BeautifulSoup

def parse_html_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    output = ""
    for dt, dd in zip(soup.find_all('dt'), soup.find_all('dd')):
        title = dt.text.strip()
        description = dd.p.text.strip()
        output += f"{title}: {description}\n\n"
    return output


# ***Convert Data to Embedding Vectors***

In [4]:
# Convert text chunks to embedding vectors
chunk_texts = [chunk['text'] for chunk in all_chunks]
chunk_embeddings = model.encode(chunk_texts)


In [5]:
pip install grpcio==1.63.0

In [6]:
pip install faiss-cpu

## **Create Vector Database **

In [7]:
import faiss
import numpy as np

# Convert the embeddings to float32
chunk_embeddings = np.array(chunk_embeddings).astype('float32')

# Create a FAISS index
dimension = chunk_embeddings.shape[1]
index = faiss.IndexHNSWFlat(dimension, 32)  # HNSW index with 32 neighbors
index.hnsw.efConstruction = 200

# Add embeddings to the index
index.add(chunk_embeddings)

# Store metadata (in-memory for simplicity, you may use a database or file)
metadata = [{'web_link': chunk['meta']['web_link']} for chunk in all_chunks]


# ** Retrieval and Re-ranking**

In [8]:
from transformers import pipeline

# Use a simple query expansion technique (e.g., synonyms)
def expand_query(query):
    synonyms = ["synonym1", "synonym2"]  # Example synonyms
    expanded_query = f"{query} {' '.join(synonyms)}"
    return expanded_query

# Expand the query
query = "example query"
expanded_query = expand_query(query)


# **Hybrid Retrieval Methods**

In [9]:

# Encode the query
query_embedding = model.encode([expanded_query]).astype('float32')

# Search the index
k = 10  # Number of nearest neighbors
distances, indices = index.search(query_embedding, k)

# Extract results
retrieved_chunks = [chunk_texts[i] for i in indices[0]]
retrieved_metadata = [metadata[i] for i in indices[0]]

print(f"Retrieved Chunks: {retrieved_chunks}")
print(f"Retrieved Metadata: {retrieved_metadata}")


Retrieved Chunks: ['This guide summarizes the ways that applications can be fine-tuned to gain additional speedups by leveraging Hopper GPU Architectureâ\x80\x99s features.</p>\r\n</dd>\r\n<dt><a class="reference internal" href="ada-tuning-guide/index.html"><span class="doc">Ada Tuning Guide</span></a></dt><dd><p>The NVIDIAÂ® Ada GPU architecture is NVIDIAâ\x80\x99s latest architecture for CUDAÂ® compute applications This guide summarizes the ways that applications can be fine-tuned to gain additional speedups by leveraging NVIDIA Ampere GPU Architectureâ\x80\x99s features.</p>\r\n</dd>\r\n<dt><a class="reference internal" href="hopper-tuning-guide/index.html"><span class="doc">Hopper Tuning Guide</span></a></dt><dd><p>Hopper GPU Architecture is NVIDIAâ\x80\x99s 9th-generation architecture for CUDA compute applications This guide summarizes the ways that applications can be fine-tuned to gain additional speedups by leveraging Turing architectural features.</p>\r\n</dd>\r\n<dt><a class=

In [10]:
# Re-rank the retrieved results
reranked_results = util.semantic_search(model.encode(query), model.encode(retrieved_chunks), top_k=10)

# Flatten the re-ranked results
reranked_chunks = [retrieved_chunks[idx['corpus_id']] for idx in reranked_results[0]]


In [11]:
from transformers import pipeline

# Initialize QA pipeline
qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2')


In [12]:
# Use the top re-ranked result for question answering
context = reranked_chunks[0]
answer = qa_pipeline({'question': query, 'context': context})

print(f"Answer: {answer['answer']}")


Answer: NVIDIA Ampere GPU Architecture


In [13]:
pip install streamlit

In [14]:
! pip install streamlit -q

In [15]:
!wget -q -O - ipv4.icanhazip.com

34.125.202.28


In [17]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.202.28:8501

npx: installed 22 in 2.167s
your url is: https://curly-foxes-smile.loca.lt
2024-07-16 17:03:18.451091: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 17:03:18.451152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 17:03:18.452654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 17:03:19.793329: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find